In [1]:
cd ..

/Users/nofar/Dropbox (Weizmann Institute)/Nofar Azulay’s files/Home/Code/mibi-bin-tools_to_use/mibi-bin-tools


In [2]:
# import required packages
import os
import json
import mmap
import multiprocessing as mp
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import datetime
import warnings
from glob import glob
from matplotlib.lines import Line2D

from mibi_bin_tools import bin_files, io_utils

plt.rcParams["figure.figsize"] = (20,13)
plt.rcParams["ytick.color"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["axes.labelcolor"] = "black"
plt.rcParams["axes.edgecolor"] = "w"
plt.rcParams["axes.facecolor"] = 'black'
plt.rcParams["savefig.edgecolor"] = "w"
plt.rcParams["savefig.facecolor"] = 'black'
plt.rcParams["figure.facecolor"] = 'black'
plt.rcParams["figure.constrained_layout.use"] = False
plt.rcParams["legend.facecolor"] = 'w'
warnings.filterwarnings("ignore")

### Add a folder to '/data' with bin and jason files for all fovs

In [3]:
# set up file paths
base_dir = "data/test"
data_dir = base_dir

extracted_dir = os.path.join(base_dir, 'extracted')

In [4]:
# create directories if do not exist
for directory in [extracted_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# 1 . Calibrate spectrum

### You can skip this step and go to extraction if you want to use automatic calibration

In [5]:
#select fov (only one) to extract spectrum
include_fovs = ['fov-1-scan-1']

In [6]:
#extracts and saves spectrum.csv before calibration
spectra_data = bin_files.extract_spectra(data_dir, os.path.join(extracted_dir , 'before_calibration'), include_fovs=include_fovs)

### Plot spectrum before calibration

In [7]:
%matplotlib tk

fig, axs = plt.subplots(1, 2 , constrained_layout=True )
axs[0].plot(spectra_data['time_offset'] , spectra_data['Counts'] ,'tab:green' , label = 'Spectrum in time')
axs[0].legend(loc="upper right")
axs[0].set_ylabel('Counts')
axs[0].set_xlabel('Time [A/D units]')
axs[1].plot(spectra_data['massList'] ,spectra_data['Counts'] ,'tab:blue', label = 'Spectrum in mass (auto calibration)')
axs[1].set_ylabel('Counts')
axs[1].set_xlabel('Mass [dalton]')
axs[1].legend(loc="upper right")
axs[1].vlines(x = [22.98976928,196.966], ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)
axs[0].grid(linewidth=0.7)
axs[1].grid(linewidth=0.7)

### Insert values based on spectra (m1,t1) , (m2, t2)

In [8]:
m1 = 22.98976928
m2 = 196.966
t1 = 1044.6
t2 = 13293.5

In [9]:
#get automatic calibration parameters
with open(os.path.join(data_dir, include_fovs[0] + '.json'), 'rb') as f:
    data = json.load(f)

mass_gain_old = data['fov']['fullTiming']['massCalibration']['massGain']
mass_offset_old = data['fov']['fullTiming']['massCalibration']['massOffset']

In [10]:
#get new calibration parameters
mass_offset , mass_gain = bin_files.calibrate_spectrum(t1 , m1 , t2 , m2 , mass_gain_old , mass_offset_old)
spectra_data['mass_cal'] = pd.Series(bin_files.tof2mass(spectra_data['time_offset'].to_numpy(), mass_offset, mass_gain))


In [11]:
#save parameters to log file
with open(os.path.join(extracted_dir ,'calibration_param.log'), 'w') as f:
    f.write(str(datetime.datetime.now()))
    f.write('\n')
    f.write('fov_name = ' + include_fovs[0])
    f.write('\n')
    f.write('mass_gain = ' + str(mass_gain))
    f.write('\n')
    f.write('mass_offset = ' + str(mass_offset))
    f.write('\n')
    f.write('(m1 , t1) = (%f , %f)' %(m1 , t1))
    f.write('\n')
    f.write('(m2 , t2) = (%f , %f)' %(m2 , t2))

In [12]:
#extracts and saves calibrated spectrum.csv
bin_files.extract_spectra(data_dir, os.path.join(extracted_dir , 'after_calibration'), include_fovs=include_fovs , calibration = (mass_offset , mass_gain));

### Plot new vs. old calibration

In [13]:
%matplotlib tk

fig, axs = plt.subplots(1, 2 , constrained_layout=True , sharex=True , sharey=True)
axs[0].plot(spectra_data['mass_cal'] , spectra_data['Counts'] ,'tab:green' , label = 'Spectrum in mass (maual calibration)')
axs[0].set_title('New calibration')
axs[0].set_ylabel('Counts')
axs[0].set_xlabel('Time [A/D units]')
axs[0].legend(loc="upper right")
axs[0].vlines(x = [m1,m2], ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)
axs[1].plot(spectra_data['massList'] ,spectra_data['Counts'] ,'tab:blue', label = 'Spectrum in mass (auto calibration)')
axs[1].set_ylabel('Counts')
axs[1].set_xlabel('Mass [dalton]')
axs[1].set_title('current calibration')
axs[1].legend(loc="upper right")
axs[1].vlines(x = [m1,m2], ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)
axs[0].grid(linewidth=0.7)
axs[1].grid(linewidth=0.7)

# 2. Plot multiple spectra to determine integration windows

### You can skip this step and go to extraction if you have already determined integration windows

#plot_spectra - 'auto' plot spectra using the machine calibration , 'manual' plot spectra after manual calibration

In [15]:
#plot_spectra = 'auto'
plot_spectra = 'manual'

masses_list = pd.read_csv('templates/accurate_masses.csv')
masses_list = masses_list['Mass'].values

%matplotlib tk
if plot_spectra == 'auto':
    spectra_files = glob(os.path.join(data_dir, "*spectrum.csv"))
    for file in spectra_files:
        df = pd.read_csv(file)
        plt.plot(df['m/z'] , df['count'], label = str.split(file , '/')[-1])
        plt.xlim((0 , 210))
        plt.xlabel("Mass")
        plt.ylabel("Counts")
        plt.vlines(x = masses_list, ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)
        plt.grid(linewidth=0.7)
        handles, labels = plt.gca().get_legend_handles_labels()  
        line_accurate = Line2D([0], [0], label='accurate mass', color='r')
        handles.extend([line_accurate])
    plt.legend(handles=handles)
    
elif plot_spectra == 'manual':
    #extracts and saves calibrated spectrum.csv
    include_fovs = None
    bin_files.extract_spectra(data_dir, os.path.join(extracted_dir , 'after_calibration'), include_fovs=include_fovs , calibration = (mass_offset , mass_gain))
    spectra_files = glob(os.path.join(os.path.join(extracted_dir , 'after_calibration' , 'spectrum_files'), "*.csv"))
    for file in spectra_files:
        df = pd.read_csv(file)
        plt.plot(df['massList'] , df['Counts'], label = str.split(file , '/')[-1])
        plt.xlim((0 , 210))
        plt.xlabel("Mass")
        plt.ylabel("Counts")
        plt.vlines(x = masses_list, ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)
        plt.grid(linewidth=0.7)
        handles, labels = plt.gca().get_legend_handles_labels()  
        line_accurate = Line2D([0], [0], label='accurate mass', color='r')
        handles.extend([line_accurate])
    plt.legend(handles=handles)
    

# 3. Extract bin files

#include_foves - list of fovs to extract, if 'None' all bin files in folder are extracted.

#panel - tuple of integration window or csv file with 'Start' and 'Stop' columns.

#calibration - tuple of (mass offset, mass_gain) , if 'auto' using the machine parameters.

In [16]:
#include_fovs = ['fov-1-scan-1']
include_fovs = None

#panel = (-0.3, 0.3)
panel = pd.read_csv(os.path.join(base_dir,'panel.csv'))

#calibration = 'auto'
calibration = (mass_offset , mass_gain)

In [17]:
bin_files.extract_bin_files(data_dir, extracted_dir, include_fovs=include_fovs, panel=panel , calibration = calibration)